# Structural Analysis

## Importing Libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import json

## Inputs

In [2]:
MAC = 0.4 # Mean Aerodynamic Chord [m]
AR = 14 # [-]
S = 5.5 # [m^2]
taper = 0.4 # [-]
rootchord = 0.5 # [m]
thicknessChordRatio = 0.17 # [-]
xAC = 0.25 # [-] position of ac with respect to the chord
# Ldstr = 
mtom = 1972 # maximum take-off mass from statistical data - Class I estimation
S1, S2 = 5.5, 5.5 # surface areas of wing one and two
A = 14 # aspect ratio of a wing, not aircraft
nmax = 3.2 # maximum load factor
Pmax = 15.25 # this is defined as maximum perimeter in Roskam, so i took top down view of the fuselage perimeter
lf = 7.2 # length of fuselage
m_pax = 95 # average mass of a passenger according to Google
n_prop = 16 # number of engines
n_pax = 5 # number of passengers (pilot included)
pos_fus = 3.6 # fuselage centre of mass away from the nose
pos_lgear = 3.6 # landing gear position away from the nose
pos_frontwing, pos_backwing = 0.2, 7 # positions of the wings away from the nose
m_prop = [30]*16 # list of mass of engines (so 30 kg per engine with nacelle and propeller)
pos_prop = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0] # 8 on front wing and 8 on back wing
Mac = 0.0645 * 0.5 * 1.2 * 53 ** 2 * 5.25 * 0.65 # aerodynamic moment around AC
flighttime = 3 # [hr]
turnovertime = 2 # we dont actually need this xd
takeofftime = 0.1 # take off time
span = np.sqrt(AR * S)
enginePlacement = list(np.linspace(0.1*span/2, 0.8*span/2, 4))
engineMass = 400 * 9.81 / 8
Thover = 2960
Tcruise = 2960

## Optimized

In [3]:
thicknessOfSkin = 1e-3 # [m]
thicknessOfSpar = 1e-2 # [m]
nStrT = 1  # [-] number of stringers on the top
nStrB = 1 # [-] number of stringers on the bottom
StrA = 0.001 # [m^2] stringer cross-sectional area


## Wing Structure Geometry

In [6]:
from Geometry import HatStringer, JStringer, ZStringer, WingBox, WingStructure
from SolveLoads import WingLoads, Engines, Fatigue
from Weight import *
from Material import Material

base, height = 0.75 - 0.15, 0.11571117509557907 + 0.03145738125495376 # x/c, y/c

n_ult = nmax*1.5 # 3.2 is the max we found, 1.5 is the safety factor

print('normalBox =', normalBox := WingBox(thicknessOfSkin, thicknessOfSpar, base, height))

print('wing =', wing := WingStructure(span := (AR * S) ** 0.5, taper, rootchord, normalBox))

hatGeom = dict(bflange1 = 0.02, bflange2 =0.02, tflange = 0.02, vflange = 0.035, tstr = 0.001)

args = dict(span=span, taper=taper, cr=rootchord, tsk=thicknessOfSkin, tsp=thicknessOfSpar,
            toc=thicknessChordRatio, nStrT=nStrT, nStrB=nStrB, StrA=StrA, strType='Hat', strGeo=hatGeom, mac=MAC, xac=xAC,
            engines=Engines(Thover, Tcruise, enginePlacement, engineMass),
           frac=0.6)

loads = WingLoads(**args)

wing = Wing(mtom, S1, S2, n_ult, A, [pos_frontwing, pos_backwing])
fuselage = Fuselage(mtom, Pmax, lf, n_pax, pos_fus)
lgear = LandingGear(mtom, pos_lgear)
props = Propulsion(n_prop, m_prop, pos_prop)
w = Weight(m_pax, wing, fuselage, lgear, props, cargo_m = 85, cargo_pos = 6, battery_m = 400, battery_pos = 3.6, p_pax = [1.5, 3, 3, 4.2, 4.2])

wingWeight = w.wing.mass[0] * 9.81
lift = nmax * 1.5 * w.mtom * 9.81
drag = lift / 19.03 # 19.03 is lift-to-drag ratio, need to change (replace by drag distribution)
pos = np.linspace(0, span/2, 10000)
dragd = 2 * drag / (np.pi * span) * np.sqrt(1 - 4 * np.power(pos / span, 2))
liftd = 2 * lift / (np.pi * span) * np.sqrt(1 - 4 * np.power(pos / span, 2))

reactionsCruise = loads.equilibriumCruise([pos, dragd], [pos, liftd], [pos, [Mac / span]*len(pos)], wingWeight)
reactionsVTO = loads.equilibriumVTO(wingWeight)
cruise_loads = loads.internalLoads([pos, dragd], [pos, liftd], [pos, [Mac / span]*len(pos)], wingWeight)
VxVTO, MyVTO = loads.internalLoadsVTO(wingWeight)
with open('ldists.json', 'w') as ld:
    dists = {'liftd': [float(l) for l in liftd], 'dragd': [float(d) for d in dragd], 'pos': [float(p) for p in pos]}
    ld.write(json.dumps(dists, indent=3))

normalBox = Wingbox(Height=0.14716855635053283, Base=0.6, Tsk = 0.001, Tsp = 0.01, Stringers = 0)
wing = WingStructure(span=8.774964387392123, taper=0.4, rc=0.5, tc=0.2, box=Wingbox(Height=0.14716855635053283, Base=0.6, Tsk = 0.001, Tsp = 0.01, Stringers = 0))


In [ ]:
from Draw import InternalLoading
InternalLoading(0, span/2, **{l: loads[l] for l in 'T, My, Mx, Vx, Vy'.split(', ')}).show(renderer='iframe')

In [ ]:
InternalLoading(0, span/2, Vx = VxVTO, My = MyVTO).show(renderer='iframe')

In [ ]:
coords, o_cr, tau_cr, Ymcr = loads.stressesCruise()
coords, o_VTO, tau_VTO, YmVTO = loads.stressesVTO()

In [ ]:
# cruise
print(WingLoads.extreme(coords, o_cr), WingLoads.extreme(coords, tau_cr))
print(WingLoads.extreme(coords, Ymcr))
print(sigmacr_max := WingLoads.extreme(coords, o_cr)[2]*1e-6, taucr_max := WingLoads.extreme(coords, tau_cr)[2]*1e-6)

In [ ]:
# VTOL
print(WingLoads.extreme(coords, o_VTO), WingLoads.extreme(coords, tau_VTO))
print(WingLoads.extreme(coords, YmVTO))
print(sigmaVTO_max := WingLoads.extreme(coords, o_VTO)[2]*1e-6, tauVTO_max := WingLoads.extreme(coords, tau_VTO)[2]*1e-6)

In [ ]:

# material = Material.load(file =material='Al 6061', Condition='T6')
import os

basepath =  '../' if 'structures' in os.popen('pwd').read().strip('\n') else ''
material = Material.load(file = basepath + 'data/materials.csv', material='Al 6061', Condition='T6') 

In [ ]:
# fatigue

lift = 1 * 1.5 * w.mtom * 9.81
drag = lift / 19.03 # 19.03 is lift-to-drag ratio, need to change (replace by drag distribution)
pos = np.linspace(0, span/2, 10000)
dragd = 2 * drag / (np.pi * span) * np.sqrt(1 - 4 * np.power(pos / span, 2))
liftd = 2 * lift / (np.pi * span) * np.sqrt(1 - 4 * np.power(pos / span, 2))

fatigue_reactionsCruise = loads.equilibriumCruise([pos, dragd], [pos, liftd], [pos, [Mac / span]*len(pos)], wingWeight)
fatigue_lift, fatigue_wgt = loads.internalLoads([pos, dragd], [pos, liftd], [pos, [Mac / span]*len(pos)], wingWeight)
coords, ocrf, taucrf, Ymcrf = loads.stressesCruise()

fatigue_reactionsVTO = loads.equilibriumVTO(wingWeight)
fatigue_VxVTO, fatigue_MyVTO = loads.internalLoadsVTO(wingWeight)
coords, oVTOf, tauVTOf, YmVTOf = loads.stressesVTO()

fatigue_reactionsVTOgr = loads.equilibriumVTO(wingWeight, ground = True)
fatigue_VxVTOgr, fatigue_MyVTOgr = loads.internalLoadsVTO(wingWeight, ground = True)
coords, oVTOfgr, tauVTOfgr, YmVTOfgr = loads.stressesVTO()

*coor, maxDif = loads.extreme(coords, oVTOf - ocrf)
ind = [i for i in range(len(coords)) if np.all(coords[i] == coor)][0]

oVTOfgrmd, oVTOfmd, ocrfmd = oVTOfgr[ind], oVTOf[ind], ocrf[ind]

fatigue = Fatigue(oVTOfgrmd, oVTOfmd, ocrfmd, flighttime, turnovertime, takeofftime, material)

t, y = fatigue.determineCycle()
fdf = fatigue.getCycles()
print(fatLife := fatigue.MinersRule())

In [ ]:
from Draw import DrawFatigue
DrawFatigue(t, y).show(renderer='iframe')

In [ ]:
root = loads.wing(0)

EofStringers = 72e9
vOfStringers = 0.33
yieldOfStringers = 275e6
EofSkin = 72e9
vOfSkin = 0.33

root.Bstress(EofStringers, vOfStringers, yieldOfStringers, EofSkin, vOfSkin)*1e-6
